# Revision Code

In [1]:
using LinearAlgebra, ColorBitstring, Plots, Test
import Base: getindex, setindex, size, *, \

## Lab 4
`mul_cols` for dense matrices:

In [2]:
function mul_cols(A,x)
    m,n = size(A)
    b = zeros(eltype(x), m)
    for j=1:n, k=1:m
        b[k] = A[k,j]*x[j]
    end
    b
end


mul_cols (generic function with 1 method)

In [3]:
zeros(Int, 5)
fill(0.0, 5)
[0 for k=1:5]

5-element Vector{Int64}:
 0
 0
 0
 0
 0

In [4]:
ones(Int, 5, 6)
fill(1, 5, 6)
[1 for k=1:5, j=1:6]

5×6 Matrix{Int64}:
 1  1  1  1  1  1
 1  1  1  1  1  1
 1  1  1  1  1  1
 1  1  1  1  1  1
 1  1  1  1  1  1

In [5]:
# Note: (1:5)' is a "row-vector" which behaves differently than a matrix
Matrix((1:5)')

# 2. Broadcast:
#exp.(-(1:5))

# 3. Explicit broadcsat:
#broadcast(k -> exp(-k), 1:5)

# 4. Comprehension:
#[exp(-k) for k=1:5]

1×5 Matrix{Int64}:
 1  2  3  4  5

More on broadcasting

In [6]:
k = 1:5
j = 1:6
cos.(k .+ j')

5×6 Matrix{Float64}:
 -0.416147  -0.989992  -0.653644   0.283662   0.96017    0.753902
 -0.989992  -0.653644   0.283662   0.96017    0.753902  -0.1455
 -0.653644   0.283662   0.96017    0.753902  -0.1455    -0.91113
  0.283662   0.96017    0.753902  -0.1455    -0.91113   -0.839072
  0.96017    0.753902  -0.1455    -0.91113   -0.839072   0.0044257

In [7]:
broadcast((k,j) -> cos(k+j), 1:5, (1:6)')

5×6 Matrix{Float64}:
 -0.416147  -0.989992  -0.653644   0.283662   0.96017    0.753902
 -0.989992  -0.653644   0.283662   0.96017    0.753902  -0.1455
 -0.653644   0.283662   0.96017    0.753902  -0.1455    -0.91113
  0.283662   0.96017    0.753902  -0.1455    -0.91113   -0.839072
  0.96017    0.753902  -0.1455    -0.91113   -0.839072   0.0044257

Triangular matrices `mul_cols`:

In [8]:
function mul_cols(U::UpperTriangular, x)
    n = size(U,1)
    T = promote_type(eltype(x), eltype(U))
    b = zeros(T,n)
    for j = 1:n, k = 1:j
        b[k] += U[k,j] * x[j]
    end
    b
end

function mul_cols(L::LowerTriangular, x)
    n = size(L,1)
    T = promote_type(eltype(x), eltype(L))
    b = zeros(T,n)
    for j = 1:n, k = j:n
        b[k] += L[k,j] * x[j]
    end
    b
end

mul_cols (generic function with 3 methods)

Solving linear systems:

In [9]:
function ldiv(L::LowerTriangular, b)
    n = size(L, 1)
    T = promote_type(eltype(L), eltype(b))
    if length(b) != n
        error("The system is not compatible")
    end
    x = zeros(T, n)

    for k = 1:n
        r = b[k]
        for j = 1:k-1
            r -= L[k,j]*x[j]
        end
        x[k] = r/L[k,k]
    end
    # END
    x
end


function ldiv(U::UpperTriangular, b)
    n = size(L, 1)
    T = promote_type(eltype(U), eltype(b))
    if length(b) != n
        error("The system is not compatible")
    end
    x = zeros(T, n)

    for k = n:-1:1
        r = b[k]
        for j = n:-1:k+1
            r -= U[k,j]*x[j]
        end
        x[k] = r/U[k,k]
    end
    # END
    x
end

L = LowerTriangular(randn(5,5))
b = randn(5)
@test L\b ≈ ldiv(L, b)

U = UpperTriangular(randn(5,5))
b = randn(5)
@test U\b ≈ ldiv(U, b)

Test Passed

Upper Tridiagonal matrices:

In [10]:
struct UpperTridiagonal{T} <: AbstractMatrix{T}
    d::Vector{T}   # diagonal entries: d[k] == U[k,k]
    du::Vector{T}  # super-diagonal enries: du[k] == U[k,k+1]
    du2::Vector{T} # second-super-diagonal entries: du2[k] == U[k,k+2]
end

size(U::UpperTridiagonal) = (length(U.d),length(U.d))

function getindex(U::UpperTridiagonal, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if j == k+2
    	return U.du2[k]
    elseif j == k+1
    	return U.du[k]
    elseif j == k
    	return U.d[k]
    else # off band entries are zero
    	return zero(eltype(U))
    end
end

function setindex!(U::UpperTridiagonal, v, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if j > k+2 || j < k
        error("Cannot modify off-band")
    end

    if j == k+2
    	du2[k] = v
    elseif j == k+1
    	du[k] = v
    elseif j == k
    	d[k] = v
    end
    U # by convention we return the matrix
end

function *(U::UpperTridiagonal, x::AbstractVector)
    n = size(U,1)
    T = promote_type(eltype(x),eltype(U))
    b = zeros(T, n)
    for j = 1:n, k = max(j-2,1):j
        b[k] += U[k, j] * x[j]
    end
    b
end

function \(U::UpperTridiagonal, b::AbstractVector)
    n = size(U,1)
    T = promote_type(eltype(b),eltype(U))

    if length(b) != n
        error("The system is not compatible")
    end

    x = zeros(T, n)
    for k = n:-1:1  # start with k=n, then k=n-1, ...
        r = b[k]  # dummy variable
        for j = k+1:min(n, k+2)
            r -= U[k,j]*x[j] # equivalent to r = r - U[k,j]*x[j]
        end
        # after this for loop, r = b[k] - ∑_{j=k+1}^n U[k,j]x[j]
        x[k] = r/U[k,k]
    end
    x
end

n = 1_000_000 # under-scores are like commas: so this is a million: 1,000,000
U = UpperTridiagonal(ones(n), fill(0.5,n-1), fill(0.1,n-2))
x = ones(n)
b = [fill(1.6,n-2); 1.5; 1] # exact result
# note following should take much less than a second
@test U*x ≈ b
@test U\b ≈ x

Test Passed

## Lab 5
Permutation matrices:

In [11]:
struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int}
    function PermutationMatrix(p::Vector)
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p)
    end
end

function size(P::PermutationMatrix)
    (length(P.p),length(P.p))
end


function getindex(P::PermutationMatrix, k::Int, j::Int)
    P.p[k] == j ? 1 : 0
end
function *(P::PermutationMatrix, x::AbstractVector)
    x[P.p]
end

p = [1, 4, 2, 5, 3]
P = PermutationMatrix(p)
@test P == I(5)[p,:]

n = 100_000
p = Vector(n:-1:1)
P = PermutationMatrix(p)
x = randn(n)
@test P*x == x[p]

Test Passed

Reflection:

In [12]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

function size(Q::Reflection)
    (length(Q.v),length(Q.v))
end

function getindex(Q::Reflection, k::Int, j::Int)
    if k == j
        1 - 2Q.v[k]*conj(Q.v[j])
    else
        - 2Q.v[k]*conj(Q.v[j])
    end
end

function *(Q::Reflection, x::AbstractVector)
    x - 2*Q.v * dot(Q.v,x) # (Q.v'*x) also works instead of dot
end

function householderreflection(s::Bool, x::AbstractVector)
    y = copy(x) # don't modify `x`
    if s
        y[1] -= norm(x)
    else
        y[1] += norm(x)
    end
    Reflection(y)
end

householderreflection (generic function with 1 method)

Complete the definition of `Reflections` which supports a sequence of reflections,
that is,
$$
Q = Q_{𝐯_1} ⋯ Q_{𝐯_m}
$$
where the vectors are stored as a matrix $V ∈ ℂ^{n × m}$ whose $j$-th column is $𝐯_j∈ ℂ^n$, and
$$
Q_{𝐯_j} = I - 2 𝐯_j 𝐯_j^⋆
$$
is a reflection.

In [13]:
struct Reflections{T} <: AbstractMatrix{T}
    V::Matrix{T}
end

size(Q::Reflections) = (size(Q.V,1), size(Q.V,1))


function *(Q::Reflections, x::AbstractVector)
    m,n = size(Q.V)
    for j = n:-1:1
        x = Reflection(Q.V[:, j]) * x
    end
    x
end

function getindex(Q::Reflections, k::Int, j::Int)
    T = eltype(Q.V)
    m,n = size(Q)
    ej = zeros(T, m)
    ej[j] = one(T)
    return (Q*ej)[k]
end

Y = randn(5,3)
V = Y * Diagonal([1/norm(Y[:,j]) for j=1:3])
Q = Reflections(V)
@test Q ≈ (I - 2V[:,1]*V[:,1]')*(I - 2V[:,2]*V[:,2]')*(I - 2V[:,3]*V[:,3]')
@test Q'Q ≈ I

Test Passed

## Lab 6
Vandermonde matrix:

In [14]:
function vandermonde(𝐱, n) # 𝐱 = [x_1,…,x_m]
    m = length(𝐱)
    # 𝐱 .^ (0:n-1)'
    [𝐱[j]^k for j = 1:m, k = 0:n-1]
end

vandermonde (generic function with 1 method)

Interpolation:

In [15]:
n = 50
𝐱 = range(-1, 1; length=n)
𝐠 = range(-1, 1; length=1000)

# TODO: interpolate 1/(10x^2 + 1) and 1/(25x^2 + 1) at $𝐱$, plotting both solutions evaluated at the grid 𝐠. 
# Use a rectangular Vandermonde matrix to evaluate your polynomial on 𝐠. Remember `plot(𝐱, 𝐟)` will create
# a new plot whilst `plot!(𝐱, 𝐟)` will add to an existing plot.

V = vandermonde(𝐱, n)
V_g = vandermonde(𝐠, n)
f_4 = x -> 1/(4x^2 + 1)
𝐜_4 = V \ f_4.(𝐱)
f_25 = x -> 1/(25x^2 + 1)
𝐜_25 = V \ f_25.(𝐱)

plot(𝐠, V_g*𝐜_4; ylims=(-1,1))
plot!(𝐠, V_g*𝐜_25)

Least square version:

In [ ]:
n = 50 # use basis [1,x,…,x^(49)]
𝐱 = range(-1, 1; length=500) # least squares grid
𝐠 = range(-1, 1; length=2000) # plotting grid

V = vandermonde(𝐱, n)
V_g = vandermonde(𝐠, n)
f_4 = x -> 1/(4x^2 + 1)
𝐜_4 = V \ f_4.(𝐱)
f_25 = x -> 1/(25x^2 + 1)
𝐜_25 = V \ f_25.(𝐱)

plot(𝐠, V_g*𝐜_4; ylims=(-1,1))
plot!(𝐠, V_g*𝐜_25)

Householder QR:

In [ ]:
function householderqr(A)
    T = eltype(A)
    m,n = size(A)
    if n > m
        error("More columns than rows is not supported")
    end

    R = zeros(T, m, n)
    Q = Reflections(zeros(T, m, n))
    Aⱼ = copy(A)

    for j = 1:n
        # TODO: rewrite householder QR to use Reflection and
        # Reflections, in a way that one achieves O(mn^2) operations
        # SOLUTION
        𝐚₁ = Aⱼ[:,1] # first columns of Aⱼ
        Q₁ = householderreflection(𝐚₁[1] < 0, 𝐚₁)
        Q₁Aⱼ = Q₁*Aⱼ
        α,𝐰 = Q₁Aⱼ[1,1],Q₁Aⱼ[1,2:end]
        Aⱼ₊₁ = Q₁Aⱼ[2:end,2:end]

        # populate returned data
        R[j,j] = α
        R[j,j+1:end] = 𝐰

        Q.V[j:end, j] = Q₁.v

        Aⱼ = Aⱼ₊₁ # this is the "induction"
        # END
    end
    Q,R
end

Note in Julia `opnorm(A)` is the induced matrix 2-norm. `norm(A) == norm(vec(A))` is the Fröbenius norm.
The following code samples a function on a grid in the square `[-1,1]^2`
and plots the corresponding pixels:

In [ ]:
m,n = 150,100
x = range(-1, 1; length=n)
y = range(-1, 1; length=m)

F = f.(x', y) # equivalent to [f(x[j],y[k]) for k=1:m, j=1:n]

function fsample(f::Function, m::Int, n::Int)
    x = range(-1, 1; length=n)
    y = range(-1, 1; length=m)
    f.(x', y)
end

heatmap(x, y, F)

## Matrix compression
Use SVD to compress a matrix to its best rank-$k$ approximation:

In [ ]:
function svdcompress(A::Matrix, k::Integer)
    U,σ,V = svd(A)
    Ak = U[:,1:k]*Diagonal(σ[1:k])*V[:,1:k]'
    Ak
end

Return the smallest rank such that the rank-$k$ approximation differs by at most $\epsilon$ from the 2-norm of $A$. Note: this will be the position of the last singular value that is greater than $\epsilon$.

In [ ]:
function svdcompress_rank(A::Matrix, ε::Real)
    σ = svdvals(A)
    for k = 1:length(σ)
        if σ[k] ≤ ε
            return k-1
        end
    end
    return length(σ)
end

function svdcompress_rank2(A::Matrix, ε::Real)
    k=1
    n,m = size(A)
    while opnorm(A-svdcompress(A,k))>ε && k<min(n,m)
        k+=1
    end
    k
end

Plot of how the rank of a Hilbert matrix grows with its dimension $n$ from $1$ to $200$:

In [ ]:
hilbertmatrix(n) = [1/(k + j - 1) for k=1:n, j=1:n]

plot([svdcompress_rank(hilbertmatrix(n), 1E-10) for n=1:200], xscale=:log10)

Solving differential equation from lab 7:

In [ ]:
function forwardeuler(a, n)
    h = 1/n
    d = [1.0; fill(1/h, n)]
    dl = fill(-(a+1/h), n)
    Bidiagonal(d,dl,:L)
end

Conditional numbers:

In [ ]:
A = [1 2 3;
     3 4 5;
     1 5 2]
opnorm(A)*opnorm(inv(A)) ≈ cond(A)

In [ ]:
plot([cond(forwardeuler(1, n)) for n = 1:200]; yscale=:log10, xscale=:log10, label="cond")
plot!(1:200; label="linear")
plot!((1:200) .^ (3/2); label="quadratic")

# appears to be between $Cn$ and $Cn^2$ growth. This means we expect errors to grow
# as fast as $ϵ_{\rm m} n$.